In [1]:
#goal: augment big_table with new columns (from original one) (don't forget to create .csv with topcat)
from astropy.io import fits
import numpy as np
import os.path
import matplotlib.pyplot as plt
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy
import pandas as pd
from photutils import SkyCircularAperture
from photutils import aperture_photometry

In [2]:
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table_orig.fits'
big_table = fits.open(path)[1].data
#X-ray counts & hardness ratio & signal to noise ratio
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/tbl04.fits'
cat7ms = fits.open(path)[1].data
soft_count = np.zeros(len(cat7ms))  
hard_count = np.zeros(len(cat7ms)) 
soft_error = np.zeros(len(cat7ms)) 
hard_error = np.zeros(len(cat7ms))
snratio_soft = np.zeros(len(cat7ms)) 
snratio_hard = np.zeros(len(cat7ms))
hratio = np.zeros(len(cat7ms))
hr_error = np.zeros(len(cat7ms))
hratio2 = np.zeros(len(cat7ms))
soft_flux = np.zeros(len(cat7ms))
hard_flux = np.zeros(len(cat7ms))
full_flux = np.zeros(len(cat7ms))
for i in range(len(cat7ms)):
    soft_count[i] = cat7ms.field('SB_COUNTS')[i]/cat7ms.field('SB_EXPTIME')[i]
    hard_count[i] = cat7ms.field('HB_COUNTS')[i]/cat7ms.field('HB_EXPTIME')[i]
    if max(cat7ms.field('SB_LE')[i], cat7ms.field('SB_UE')[i]) != -1:
        soft_error[i] = max(cat7ms.field('SB_LE')[i], cat7ms.field('SB_UE')[i])/cat7ms.field('SB_EXPTIME')[i]
    else:
        soft_error[i] = -1
    if max(cat7ms.field('HB_LE')[i], cat7ms.field('HB_UE')[i]) != -1:
        hard_error[i] = max(cat7ms.field('HB_LE')[i], cat7ms.field('HB_UE')[i])/cat7ms.field('HB_EXPTIME')[i]
    else:
        hard_error[i] = -1
    hratio2[i] = (hard_count[i]-soft_count[i])/(hard_count[i]+soft_count[i])
    if (soft_error[i] != -1) & (hard_error[i] != -1):
        hratio[i] = (hard_count[i]-soft_count[i])/(hard_count[i]+soft_count[i])
        hr_error[i] = pow(4.*pow(soft_count[i],2.)*pow(hard_error[i],2.) + 4.*pow(hard_count[i],2.)*pow(soft_error[i],2.),0.5)*pow(hard_count[i] + soft_count[i],-2.)
        snratio_soft[i] = soft_count[i]/soft_error[i]
        snratio_hard[i] = hard_count[i]/hard_error[i]
    elif soft_error[i] != -1:
        hratio[i] = (hard_count[i]-(hard_count[i]/cat7ms.field('BANDRATIO')[i]))/(hard_count[i]+(hard_count[i]/cat7ms.field('BANDRATIO')[i]))
        snratio_soft[i] = soft_count[i]/soft_error[i]
    elif hard_error[i] != -1:
        hratio[i] = ((soft_count[i]*cat7ms.field('BANDRATIO')[i])-soft_count[i])/((soft_count[i]*cat7ms.field('BANDRATIO')[i])+soft_count[i])
        snratio_hard[i] = hard_count[i]/hard_error[i]
    soft_flux[i] = cat7ms.field('SB_FLUX')[i]
    hard_flux[i] = cat7ms.field('HB_FLUX')[i]
    full_flux[i] = cat7ms.field('FB_FLUX')[i]
#goods_weightimage_mismatch
misx = np.zeros(len(big_table))
misy = np.zeros(len(big_table))
for i in range(len(big_table)):
    if (big_table.field('GOODS_sect')[i] !=0) & (i!=542):
        path = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_b' + str(i+1) + '.fits'
        im = fits.open(path)[0].header
        path2 = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_b' + str(i+1) + 'wht.fits'
        wht = fits.open(path2)[0].header
        misx[i] = im['NAXIS1']
        misy[i] = im['NAXIS2']
    if (i == 542):
        misx[i] = -1

In [3]:
#visual selection flag: #1=rejected-missing ACS/WFC3 filters #2 rejected-clearly visible in all filters #3 unsure #4-7 B,V,i,z dropouts #8 dropout later/lyman break method doesn't work #9 exceptional case  /new line each 100
#         1 2 3 4 5 6 7 8 9 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100
visual = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,
          9,2,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,3,1,1,4,1,1,1,1,1,1,1,1,2,1,1,1,2,2,4,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,2,1,1,2,2,1,1,3,2,2,1,1,1,2,1,1,2,1,1,6,2,1,2,4,2,1,2,1,4,8,6,1,1,1,1,
          2,4,2,2,4,2,1,1,2,4,1,1,1,1,2,1,2,2,2,1,2,4,1,2,2,3,1,2,2,4,2,4,1,1,2,1,2,1,6,1,1,1,8,4,4,1,2,1,1,7,1,2,1,2,2,3,2,2,2,1,2,1,3,1,1,2,6,1,2,1,1,2,1,1,2,7,2,2,3,4,1,2,2,1,2,1,5,3,1,2,1,1,2,2,2,2,2,1,2,3,
          2,2,1,2,1,1,4,2,2,7,2,2,3,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,8,4,2,2,2,2,2,1,2,2,2,2,1,2,1,2,1,2,2,2,2,1,1,2,8,8,1,7,1,1,1,1,2,2,3,2,2,1,2,5,2,1,2,2,2,2,2,5,3,2,1,2,1,1,2,2,1,6,8,2,1,2,4,6,4,2,2,6,6,1,2,2,
          2,2,1,2,5,6,2,2,1,2,4,4,2,2,1,2,2,2,1,1,4,2,2,1,2,2,1,1,3,1,2,2,4,2,2,2,2,1,2,2,2,2,2,1,1,2,2,2,1,2,2,2,8,2,2,2,1,1,1,2,1,1,2,4,7,2,1,1,2,2,4,4,1,1,2,2,2,3,1,2,2,2,2,2,1,1,2,1,4,5,2,1,1,1,1,2,2,2,2,1,
          1,2,2,2,2,2,1,2,2,2,2,2,2,2,5,2,3,2,1,1,2,4,2,4,5,2,8,1,2,7,2,1,2,3,3,1,2,5,4,2,2,1,1,2,1,8,5,7,4,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,1,2,2,2,5,1,2,2,2,2,2,5,4,1,7,2,2,2,1,2,2,2,4,2,2,1,2,1,1,2,1,4,2,1,2,
          1,2,2,2,2,2,2,2,2,2,2,1,2,1,2,4,8,2,2,1,1,8,8,1,2,1,4,2,2,1,2,2,7,2,2,2,1,8,2,6,2,1,1,2,2,1,2,2,2,2,2,2,2,2,2,2,8,2,2,1,2,5,2,2,2,2,2,2,1,4,1,2,2,2,2,2,2,1,2,2,5,2,2,2,1,1,2,7,2,2,2,2,2,4,2,2,1,2,2,2,
          2,1,2,1,2,2,2,2,2,2,2,1,2,8,5,1,6,2,2,2,1,8,2,2,2,2,2,2,1,1,1,2,4,1,2,2,4,1,5,2,2,2,2,2,2,4,2,3,2,2,2,1,1,2,2,2,2,2,2,4,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,1,4,1,2,7,1,2,2,2,2,2,2,8,1,2,2,
          2,7,1,2,7,4,2,2,2,2,2,5,2,2,4,1,6,2,1,2,2,2,2,2,2,2,1,2,1,2,2,1,2,1,2,2,1,1,2,2,2,2,1,2,2,5,4,2,2,1,7,2,1,5,1,1,2,2,8,2,4,2,1,4,1,1,1,2,1,1,2,2,1,2,1,2,2,2,1,1,2,1,2,2,8,7,2,1,1,2,1,2,1,1,1,1,2,2,1,2,
          7,2,2,1,1,2,1,1,1,2,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
          1,1,1,1,1,1,1,1]
visual_flag = np.zeros(len(big_table))
for i in range(len(big_table)):
    visual_flag[i] = visual[i]
aper = np.full((len(big_table),3), 1.)  #aper_diameter/arcsec #min_area/pixel # spitzer_aper_diameter/arcsec
for i in range(len(big_table)):
    aper[i][1]=40.
    aper[i][2]=3.
aper[100]=(1.4,30.,3.5) #new exception: was a 1: missing candels before
#aper[131]=(2.,90.,5.) #done
aper[134]=(1.4,50.,4.)
#aper[136]=(1.4,50.,5.)
aper[149]=(1.7,50.,5.)
aper[184]=(0.6,50.,14.) #from 3.5 to 0 (test)
aper[188]=(1.2,50.,2.5)
aper[193]=(0.9,50.,3.5)
aper[194]=(0.6,25.,12.5) #unwanted det. in low bands, change from 30 to 50, no det. at all-> 25
aper[195]=(1.2,50.,3.) 
aper[201]=(0.8,30.,12.5) #new source, that I was forced to include but it's shit
aper[204]=(2.0,50.,4.) 
aper[209]=(1.4,50.,3.5)
aper[221]=(1.5,50.,3.5)
aper[229]=(2.0,50.,4.) 
aper[231]=(1.4,50.,3.) #from 2.5 to 3.
aper[238]=(1.4,50.,4.) #from 3. to 3.5
aper[242]=(1.4,50.,12.5) #try to obtain flux for spitzer by changing deblending parameters
aper[243]=(1.5,50.,3.5)
aper[244]=(2.,50.,4.5)
aper[249]=(1.6,50.,4.5)
#aper[262]=(1.6,50.,0.)
aper[266]=(1.5,50.,13.)
aper[275]=(1.4,90.,13.) #false detection in z band, change from 50 to 90, try to obtain flux for spitzer
#aper[278]=(1.8,50.,3.) #change criterion from closest to brightest in photometry for this source in H band
aper[279]=(1.6,50.,4.)
aper[286]=(1.6,50.,5.)
#aper[287]=(1.3,50.,3.)
aper[306]=(1.5,50.,4.5)
aper[309]=(1.4,50.,4.5)
#aper[312]=(2.0,50.,4.) 
aper[328]=(1.2,50.,3.5)
aper[329]=(1.4,50.,12.5)
aper[352]=(1.8,50.,5.) #change deblend to get right H source from 1.6 to 1.8
aper[353]=(1.4,50.,13.5)
aper[355]=(1.2,50.,13.5)
aper[367]=(1.3,50.,4.)
aper[375]=(1.4,50.,4.)
aper[385]=(1.2,50.,12.5) #try to get spitzer data
aper[386]=(1.3,50.,4.)
aper[390]=(1.4,50.,2.5)
aper[391]=(1.0,50.,4.)
aper[392]=(1.5,40.,4.5) #from 50 to 40
aper[395]=(0.8,50.,14.5) #from 3.5 to 0 (test)
aper[396]=(1.4,30.,12.5) #try to get detection in z: from 50 to 30
aper[404]=(1.4,50.,14.)
aper[405]=(1.1,50.,3.5)
aper[410]=(1.7,50.,5.)
aper[411]=(1.4,50.,3.)
aper[420]=(2.5,50.,5.) #raise aperture size from 2 to 2.5 to get more consistent flux values
#aper[428]=(1.1,50.,3.)
aper[432]=(1.0,50.,3.)
aper[452]=(1.5,50.,4.5) #might wanna use K position instead of H 
aper[463]=(0.8,20.,12.) 
aper[464]=(1.4,50.,4.5)
aper[470]=(1.0,50.,2.5)
aper[471]=(1.0,50.,12.)
aper[488]=(1.4,50.,4.)
aper[489]=(1.2,30.,3.5) #change from 50 to 30
aper[514]=(1.7,50.,3.)
#aper[516]=(1.5,50.,3.)
aper[521]=(1.5,50.,3.5)
aper[523]=(1.4,50.,4.5)
aper[524]=(1.3,50.,3.5)
aper[526]=(1.0,50.,3.5)
aper[529]=(1.4,50.,12.) #get detection in Y band by changing deblending parameter
#aper[533]=(2.0,50.,4.)
aper[537]=(1.4,30.,4.) #50 to 30
aper[538]=(1.0,50.,4.) # done (z=4.9 redshift)
aper[545]=(1.3,50.,12.5) # problem: k band position more accurate than h: manually insert k's position in photometry
aper[546]=(1.5,50.,13.5)
aper[547]=(1.4,50.,12.) # done (high redshift)
aper[548]=(1.6,50.,4.5) 
aper[569]=(1.1,50.,4.) #from 1.5 to 1.1
aper[576]=(1.2,30.,3.5) #from 50 to 30
aper[577]=(1.4,50.,12.)
aper[579]=(1.4,50.,12.5) # done (high redshift)
aper[587]=(2.5,50.,5.5) # aper raised from 5. to 5.5, from 1.4 to 2.5
aper[596]=(1.4,50.,4.) #from 2.5 to 3.5
aper[615]=(1.5,50.,5.)
aper[616]=(1.4,50.,3.5)
aper[621]=(1.4,50.,4.)
aper[622]=(1.4,50.,4.)
aper[626]=(1.4,50.,4.)
aper[632]=(1.7,50.,5.)
aper[637]=(1.2,30.,4.) #new, check that it works
aper[639]=(0.6,30.,3.5) #from 50 to 30, check if works, from 0.7 to 0.6
aper[656]=(1.4,50.,4.)
aper[661]=(1.2,30.,3.5) # lower from 1.4 to 1.2, lower from 50 to 30, update spitzer
aper[669]=(1.6,70.,4.) #false detection in H band: raise from 50 to 90, raise aper from 1.4 to 1.6/try 90->70
aper[680]=(0.6,50.,15.) #from 5. to 0. (test)
aper[687]=(1.4,50.,13.5) # from 50 to 70, lost det, back to 50
aper[693]=(1.4,30.,4.) #from 50 to 30
aper[713]=(1.2,50.,4.5) #new, random spitzer aper
aper[714]=(1.2,50.,4.5) #from 1.4 to 1.2
aper[716]=(2.,50.,5.) #new, check that it works, force it to use H band
aper[721]=(1.4,50.,3.) 
aper[732]=(1.4,30.,4.5) #from 50 to 30
aper[736]=(1.4,30.,5.) #from 50 to 30
aper[738]=(1.4,60.,3.5) #from 50 to 60
aper[745]=(1.4,50.,4.)
#aper[747]=(1.2,70.,5.) #from 50 to 70, from 1.4 to 1.2
aper[759]=(1.4,50.,4.5)
aper[785]=(1.1,60.,15.) #from 1.4 to 1.2, to 1.0, 50 to 60
aper[788]=(1.4,50.,5.)
aper[796]=(1.2,50.,3.5) #from 1.4 to 1.2
aper[801]=(1.3,50.,5.)
aper[804]=(1.0,50.,14.) #from 1.4 to 1.2, to 1.0
aper[805]=(1.0,50.,13.5) #from 1.4 to 1.1, to 1.0
aper[811]=(0.7,50.,14.) #halve aperture from 1.4 to 0.7, make exception deblending to get correct K band
aper[814]=(1.4,30.,4.5) #from 50 to 30
aper[816]=(1.4,50.,4.5)
aper[845]=(1.4,50.,4.5)
aper[846]=(0.9,60.,12.) #from 1.4 to 1.1, to 0.9, 50 to 60
aper[850]=(1.5,60.,4.5)
aper[853]=(1.8,50.,5.) #from 1.4 to 1.8
aper[858]=(1.4,50.,4.)
aper[860]=(1.2,50.,13.) #slightly smaller aperture from 1.4 to 1.2 
aper[863]=(1.5,70.,5.) #from 50 to 70
aper[884]=(1.4,50.,4.5)
aper[885]=(1.4,70.,4.5) #from 50 to 70
aper[900]=(1.4,50.,4.)
aperture = np.zeros(len(big_table))
minarea = np.zeros(len(big_table))
spitzer_flag = np.zeros(len(big_table))
for i in range(len(aperture)):
    aperture[i] = aper[i][0]
    minarea[i] = aper[i][1]
    spitzer_flag[i] = aper[i][2]



In [4]:
#goods, candels & hugs black image flag + manual 2 -> 1,0
flag_usable = []
file_path = ['/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_b', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_i', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_v', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_z', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/candels/cutout_candels_105_', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/candels/cutout_candels_125_', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/candels/cutout_candels_160_', '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/hugs/cutout_hugs_']
for j in range(len(file_path)):
    flag_usable.append(np.zeros(len(cat7ms)))
    for i in range (len(cat7ms)):
        if ((j < 4) & (big_table.field('GOODS_sect')[i] !=0) & (i != 542)) | ((j > 3) & (j < 7)) | ((j == 7) & (big_table.field('HUGS_sect')[i] !=0)):
            path = file_path[j] + str(i+1) + '.fits'
            im = fits.open(path)[0]
            x = im.header['NAXIS1']*1.0
            y = im.header['NAXIS2']*1.0
            n_usable = 0.
            n_tot = x*y*1.0
            x1=0. 
            x2=0. 
            y1=0. 
            y2=0.
            if np.sum(im.data) == 0:
                pass
            else:
                x1 = max(0., (x/2.) - (max(x, y)/6.))
                x2 = min(x, (x/2.) + (max(x, y)/6.))
                y1 = max(0., (y/2.) - (max(x, y)/6.))
                y2 = min(y, (y/2.) + (max(x, y)/6.))
                n_tot = (int(x2)-int(x1))*(int(y2)-int(y1))*1.0
                for k in range(int(x1), int(x2)):
                    for l in range(int(y1), int(y2)):
                        if im.data[l, k] != 0:
                            n_usable += 1.
            if (n_usable/n_tot > 0.7) & (n_usable/n_tot < 0.85):
                flag_usable[j][i] = 2
            if (n_usable/n_tot >= 0.85):
                flag_usable[j][i] = 1
            print(str(i+1) + ' is done')
flag_usable.append(np.ones(len(cat7ms))) #two spitzer bands, always usable
flag_usable.append(np.ones(len(cat7ms)))

49 is done
50 is done
51 is done
52 is done
53 is done
54 is done
55 is done
56 is done
58 is done
59 is done
60 is done
61 is done
62 is done
64 is done
65 is done
66 is done
67 is done
68 is done
69 is done
70 is done
71 is done
72 is done
73 is done
74 is done
75 is done
76 is done
77 is done
78 is done
79 is done
80 is done
81 is done
83 is done
84 is done
85 is done
86 is done
87 is done
88 is done
89 is done
90 is done
91 is done
92 is done
93 is done
94 is done
95 is done
96 is done
97 is done
98 is done
100 is done
101 is done
102 is done
103 is done
104 is done
105 is done
106 is done
107 is done
108 is done
109 is done
110 is done
111 is done
112 is done
113 is done
114 is done
115 is done
116 is done
117 is done
118 is done
119 is done
120 is done
121 is done
123 is done
124 is done
126 is done
127 is done
128 is done
129 is done
130 is done
131 is done
132 is done
133 is done
134 is done
135 is done
136 is done
137 is done
138 is done
139 is done
140 is done
142 is done
144

In [5]:
#photometry parameters: flag (whether there is a source), source RA&Dec, source Mag, MagErr, Flux, FluxErr
# -> 28 new columns
a = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105_', 'candels_125_', 'candels_160_', 'hugs_', 'spitzer_ch1_','spitzer_ch2_']
b = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105', 'candels_125', 'candels_160', 'hugs', 'spitzer_ch1','spitzer_ch2']
c = ['goods', 'goods', 'goods', 'goods', 'candels', 'candels', 'candels', 'hugs', 'spitzer', 'spitzer']
phot=[]
for i in range(len(a)):
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[i] + '/cat_' + b[i] + 'full_closest.cat'
    t1 = pd.read_csv(path, delim_whitespace=True, header=None, names=['#', 'x', 'y', 'RA', 'Dec', 'MagAuto', 'MagErrAuto', 'FluxAuto', 'FluxErrAuto', 'MagAper', 'MagErrAper', 'FluxAper', 'FluxErrAper', 'ID'], usecols=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], skiprows=1)
    phot.append([np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table)), np.zeros(len(big_table))])
    for j in range(t1['#'].size):
        k = t1['ID'][j] - 1
        phot[i][0][k] = 1
        phot[i][1][k] = t1['RA'][j]
        phot[i][2][k] = t1['Dec'][j]
        phot[i][3][k] = t1['MagAuto'][j] 
        phot[i][4][k] = t1['MagErrAuto'][j]
        phot[i][5][k] = t1['FluxAuto'][j]
        phot[i][6][k] = t1['FluxErrAuto'][j]
        phot[i][7][k] = pow(10,(8.9-t1['MagAuto'][j])/2.5)
        phot[i][8][k] = t1['MagErrAuto'][j]*pow(10,(8.9-t1['MagAuto'][j])/2.5)*np.log(10)*2/5
        phot[i][9][k] = t1['MagAper'][j] 
        phot[i][10][k] = t1['MagErrAper'][j]
        phot[i][11][k] = t1['FluxAper'][j]
        phot[i][12][k] = t1['FluxErrAper'][j]
        phot[i][13][k] = pow(10,(8.9-t1['MagAper'][j])/2.5)
        phot[i][14][k] = t1['MagErrAper'][j]*pow(10,(8.9-t1['MagAper'][j])/2.5)*np.log(10)*2/5



In [6]:
#upper limit computation with photutils
fluxratio = [1.95338693e-7,9.238062e-8,1.98796897e-7,4.12286699e-7,1.12851106e-7,1.16912375e-7,1.51876003e-7,9.9999999e-10,3.92276e-6,4.80825e-6]
zeropoint = [25.673, 26.486, 25.654, 24.862, 26.2687, 26.2303, 25.9463, 31.4, 22.416, 22.195]
for j in range(len(a)):
    for i in range(len(big_table)):  
        if ((j < 8) & (visual_flag[i] > 3) & (phot[j][0][i] == 0) & (flag_usable[j][i]>0)) | ((j > 7) & (visual_flag[i] > 3) & (phot[j][0][i] == 0)):
            path = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits'
            im = fits.open(path)[0]
            if (i==100) | (i==392) | (i==452) | (i==545):
                position = SkyCoord(phot[7][1][i]*u.deg, phot[7][2][i]*u.deg, frame='icrs')
            else:
                position = SkyCoord(phot[6][1][i]*u.deg, phot[6][2][i]*u.deg, frame='icrs')
            rad=aperture[i]/2.
            if j > 7:
                if spitzer_flag[i]>10.:
                    rad=(spitzer_flag[i]-10.)/2.
                else:    
                    rad=spitzer_flag[i]/2.
            circaper = SkyCircularAperture(position, r=rad*u.arcsec)
            phot_table = aperture_photometry(im, circaper)
            if phot_table['aperture_sum'][0] < 0:
                print('negative flux ' + str(phot_table['aperture_sum'][0]) + ' for source ' + str(i+1) + ' with filter ' + b[j])
            phot[j][10][i] = zeropoint[j] - (2.5*np.log10(phot_table['aperture_sum'][0]))
            phot[j][12][i] = phot_table['aperture_sum'][0]
            phot[j][14][i] = phot_table['aperture_sum'][0]*fluxratio[j]


In [7]:
#phot-z info
path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz.zout'
cat = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

photz = np.zeros(big_table.size)
chi2 = np.zeros(big_table.size)
photz_le1 = np.zeros(big_table.size)
photz_ue1 = np.zeros(big_table.size)
photz_le2 = np.zeros(big_table.size)
photz_ue2 = np.zeros(big_table.size)
photz_le3 = np.zeros(big_table.size)
photz_ue3 = np.zeros(big_table.size)
for i in range(big_table.size):
    for j in range(cat['ID'].size):
        if cat['ID'][j] == i+1:
            photz[i] = cat['z_a'][j]
            chi2[i] = cat['chi2'][j]
            photz_le1[i] = cat['l1'][j]
            photz_ue1[i] = cat['u1'][j]
            photz_le2[i] = cat['l2'][j]
            photz_ue2[i] = cat['u2'][j]
            photz_le3[i] = cat['l3'][j]
            photz_ue3[i] = cat['u3'][j]


In [8]:
#color-color conditions
B_V = np.full(big_table.size, -99.)
V_z = np.full(big_table.size, -99.)
V_i = np.full(big_table.size, -99.)
i_z = np.full(big_table.size, -99.)
B_V_arrowup = np.full(big_table.size, -99.)
V_z_arrowleft = np.full(big_table.size, -99.)
V_i_arrowup = np.full(big_table.size, -99.)
i_z_arrowleft = np.full(big_table.size, -99.)
B_V_arrowdown = np.full(big_table.size, -99.)
V_z_arrowright = np.full(big_table.size, -99.)
V_i_arrowdown = np.full(big_table.size, -99.)
i_z_arrowright = np.full(big_table.size, -99.)
B_V_star = np.full(big_table.size, -99.)
V_z_star = np.full(big_table.size, -99.)
V_i_star = np.full(big_table.size, -99.)
i_z_star = np.full(big_table.size, -99.)
V_i_star2 = np.full(big_table.size, -99.)
i_z_star2 = np.full(big_table.size, -99.)
flag1 = np.zeros(big_table.size)
flag2 = np.zeros(big_table.size)
flag3 = np.zeros(big_table.size)
flagz = np.zeros(big_table.size)
for i in range(big_table.size):
    if visual_flag[i]>3:
        if (phot[0][9][i] > 0) & (phot[1][9][i] > 0):
            B_V[i] = phot[0][9][i] - phot[1][9][i]
        if (phot[0][9][i] > 0) & (phot[1][9][i] == 0) & (phot[1][10][i] > 0):
            B_V[i] = phot[0][9][i] - phot[1][10][i]
            B_V_arrowdown[i] = phot[0][9][i] - phot[1][10][i]
        if (phot[1][9][i] > 0) & (phot[0][9][i] == 0) & (phot[0][10][i] > 0):
            B_V[i] = phot[0][10][i] - phot[1][9][i]
            B_V_arrowup[i] = phot[0][10][i] - phot[1][9][i]
        if (phot[1][9][i] > 0) & (phot[3][9][i] > 0):
            V_z[i] = phot[1][9][i] - phot[3][9][i]
        if (phot[1][9][i] > 0) & (phot[3][9][i] == 0) & (phot[3][10][i] > 0):
            V_z[i] = phot[1][9][i] - phot[3][10][i]
            V_z_arrowleft[i] = phot[1][9][i] - phot[3][10][i]
        if (phot[3][9][i] > 0) & (phot[1][9][i] == 0) & (phot[1][10][i] > 0):
            V_z[i] = phot[1][10][i] - phot[3][9][i]
            V_z_arrowright[i] = phot[1][10][i] - phot[3][9][i]
        if (phot[1][9][i] > 0) & (phot[2][9][i] > 0):
            V_i[i] = phot[1][9][i] - phot[2][9][i]
        if (phot[1][9][i] > 0) & (phot[2][9][i] == 0) & (phot[2][10][i] > 0):
            V_i[i] = phot[1][9][i] - phot[2][10][i]
            V_i_arrowdown[i] = phot[1][9][i] - phot[2][10][i]
        if (phot[2][9][i] > 0) & (phot[1][9][i] == 0) & (phot[1][10][i] > 0):
            V_i[i] = phot[1][10][i] - phot[2][9][i]
            V_i_arrowup[i] = phot[1][10][i] - phot[2][9][i]
        if (phot[2][9][i] > 0) & (phot[3][9][i] > 0):
            i_z[i] = phot[2][9][i] - phot[3][9][i]
        if (phot[2][9][i] > 0) & (phot[3][9][i] == 0) & (phot[3][10][i] > 0):
            i_z[i] = phot[2][9][i] - phot[3][10][i]
            i_z_arrowleft[i] = phot[2][9][i] - phot[3][10][i]
        if (phot[3][9][i] > 0) & (phot[2][9][i] == 0) & (phot[2][10][i] > 0):
            i_z[i] = phot[2][10][i] - phot[3][9][i] 
            i_z_arrowright[i] = phot[2][10][i] - phot[3][9][i] 
        if (B_V[i]!=-99) & (V_z[i]!=-99):
            if (B_V[i] > 1.1 + V_z[i]) & (B_V[i] > 1.1) & (V_z[i] < 1.6):
                flag1[i]=1
                flagz[i]+=4
                B_V_star[i]=B_V[i]
                V_z_star[i]=V_z[i]
            elif (B_V_arrowup[i]!=-99) | (V_z_arrowleft[i]!=-99):
                if ((B_V_arrowup[i]!=-99) & (V_z[i] < 1.6)) | ((V_z_arrowleft[i]!=-99) & (B_V[i] > 1.1)) | ((B_V_arrowup[i]!=-99) & (V_z_arrowleft[i]!=-99)):
                    flag1[i]=2
                    flagz[i]+=4
                    B_V_star[i]=B_V[i]
                    V_z_star[i]=V_z[i]
        if (V_i[i]!=-99) & (i_z[i]!=-99):
            if ((V_i[i] > 1.5 + (0.9*i_z[i])) | (V_i[i] > 2.)) & (V_i[i] > 1.2) & (i_z[i] < 1.3) & (phot[0][11][i]/phot[0][12][i] < 2.):
                flag2[i]=1
                flagz[i]+=5
                V_i_star[i]=V_i[i]
                i_z_star[i]=i_z[i]
            elif (V_i_arrowup[i]!=-99) | (i_z_arrowleft[i]!=-99):
                if ((V_i_arrowup[i]!=-99) & (i_z[i] < 1.3) & (phot[0][11][i]/phot[0][12][i] < 2.)) | ((i_z_arrowleft[i]!=-99) & (V_i[i] > 1.2) & (phot[0][11][i]/phot[0][12][i] < 2.)) | ((V_i_arrowup[i]!=-99) & (i_z_arrowleft[i]!=-99) & (phot[0][11][i]/phot[0][12][i] < 2.)):
                    flag2[i]=2
                    flagz[i]+=5
                    V_i_star[i]=V_i[i]
                    i_z_star[i]=i_z[i]
        if (V_i[i]!=-99) & (i_z[i]!=-99):
            if (i_z[i] > 1.3) & ((phot[0][11][i]/phot[0][12][i] < 2.) | (phot[1][11][i]/phot[1][12][i] < 2.)):
                flag3[i]=1
                flagz[i]+=6
                V_i_star2[i]=V_i[i]
                i_z_star2[i]=i_z[i]
            elif (i_z_arrowright[i]!=-99):
                if ((phot[0][11][i]/phot[0][12][i] < 2.) | (phot[1][11][i]/phot[1][12][i] < 2.)):
                    flag3[i]=2
                    flagz[i]+=6
                    V_i_star2[i]=V_i[i]
                    i_z_star2[i]=i_z[i]
        if flagz[i]>6:
            flagz[i]=-1


In [9]:
#stacking
arcsec_radius=0.
stack = ['bv', 'bvi', 'bviz']
path =['','','']
stacking = np.zeros(len(big_table))
bv_flag = np.zeros(len(big_table))
bvi_flag = np.zeros(len(big_table))
bviz_flag = np.zeros(len(big_table))
for i in range(len(big_table)):
    if visual_flag[i]>3:
        if (i == 244) | (i == 353) | (i == 545):
            arcsec_radius = 0.25
        elif (i == 100) | (i == 352) | (i == 390) | (i == 392) | (i == 420) | (i == 656):
            arcsec_radius = 0.3
        elif (i == 546):
            arcsec_radius = 0.4
        else:
            arcsec_radius = 0.2
        for j in range(len(stack)):
            path[j] = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/goods_stacks/cat_goods_' + stack[j] + str(i+1) + '.cat'
        bv = pd.read_csv(path[0], delim_whitespace=True, header=None, names=['#', 'x', 'y', 'RA', 'Dec', 'MagAuto', 'MagErrAuto', 'FluxAuto', 'FluxErrAuto', 'MagAper', 'MagErrAper', 'FluxAper', 'FluxErrAper'])
        bvi = pd.read_csv(path[1], delim_whitespace=True, header=None, names=['#', 'x', 'y', 'RA', 'Dec', 'MagAuto', 'MagErrAuto', 'FluxAuto', 'FluxErrAuto', 'MagAper', 'MagErrAper', 'FluxAper', 'FluxErrAper'])
        bviz = pd.read_csv(path[2], delim_whitespace=True, header=None, names=['#', 'x', 'y', 'RA', 'Dec', 'MagAuto', 'MagErrAuto', 'FluxAuto', 'FluxErrAuto', 'MagAper', 'MagErrAper', 'FluxAper', 'FluxErrAper'])
        if (i==100) | (i==392) | (i==452) | (i==545):
            d1 = ((bv['RA']-phot[7][1][i]).pow(2) + (bv['Dec']-phot[7][2][i]).pow(2)).pow(0.5)
            d2 = ((bvi['RA']-phot[7][1][i]).pow(2) + (bvi['Dec']-phot[7][2][i]).pow(2)).pow(0.5)
            d3 = ((bviz['RA']-phot[7][1][i]).pow(2) + (bviz['Dec']-phot[7][2][i]).pow(2)).pow(0.5)
        else:
            d1 = ((bv['RA']-phot[6][1][i]).pow(2) + (bv['Dec']-phot[6][2][i]).pow(2)).pow(0.5)
            d2 = ((bvi['RA']-phot[6][1][i]).pow(2) + (bvi['Dec']-phot[6][2][i]).pow(2)).pow(0.5)
            d3 = ((bviz['RA']-phot[6][1][i]).pow(2) + (bviz['Dec']-phot[6][2][i]).pow(2)).pow(0.5)
        for j in range(len(bv['FluxAper'])):
            if (bv['FluxAper'][j]/bv['FluxErrAper'][j]>1.) & (d1[j]<arcsec_radius/3600.):
                bv_flag[i]=1
                
        for j in range(len(bvi['FluxAper'])):
            if (bvi['FluxAper'][j]/bvi['FluxErrAper'][j]>1.) & (d2[j]<arcsec_radius/3600.):
                bvi_flag[i]=1
        for j in range(len(bviz['FluxAper'])):
            if (bviz['FluxAper'][j]/bviz['FluxErrAper'][j]>1.) & (d3[j]<arcsec_radius/3600.):
                bviz_flag[i]=1
        if (bv_flag[i]==1) & (bvi_flag[i]==1) & (bviz_flag[i]==1):
            stacking[i]=4.
        elif (bv_flag[i]==0) & (bvi_flag[i]==1) & (bviz_flag[i]==1):
            stacking[i]=5.
        elif (bv_flag[i]==0) & (bvi_flag[i]==0) & (bviz_flag[i]==1):
            stacking[i]=6.
        elif (bv_flag[i]==0) & (bvi_flag[i]==0) & (bviz_flag[i]==0):
            stacking[i]=7.
        else:
            stacking[i]=-1.
        #print(stacking[i])

In [10]:
#put big_table together            
new_cols = fits.ColDefs([fits.Column(name='Visual_Flag', format='I', array=visual_flag),
                        fits.Column(name='Aperture', format='D', array=aperture),
                        fits.Column(name='Min_Area', format='D', array=minarea),
                        fits.Column(name='Soft_Count', format='D', array=soft_count),
                        fits.Column(name='Hard_Count', format='D', array=hard_count),
                        fits.Column(name='Hardness_Ratio', format='D', array=hratio),
                        fits.Column(name='Hardness_Ratio_2', format='D', array=hratio2),
                        fits.Column(name='Soft_Count_Err', format='D', array=soft_error),
                        fits.Column(name='Hard_Count_Err', format='D', array=hard_error),
                        fits.Column(name='Hardness_Ratio_Err', format='D', array=hr_error),
                        fits.Column(name='SN_Ratio_Hard', format='D', array=snratio_hard),
                        fits.Column(name='SN_Ratio_Soft', format='D', array=snratio_soft),
                        fits.Column(name='Soft_Flux', format='D', array=soft_flux),
                        fits.Column(name='Hard_Flux', format='D', array=hard_flux),
                        fits.Column(name='Full_Flux', format='D', array=full_flux),
                        fits.Column(name='phot_z', format='D', array=photz),
                        fits.Column(name='Chi_2', format='D', array=chi2),
                        fits.Column(name='phot_z_l1', format='D', array=photz_le1),
                        fits.Column(name='phot_z_u1', format='D', array=photz_ue1),
                        fits.Column(name='phot_z_l2', format='D', array=photz_le2),
                        fits.Column(name='phot_z_u2', format='D', array=photz_ue2),
                        fits.Column(name='phot_z_l3', format='D', array=photz_le3),
                        fits.Column(name='phot_z_u3', format='D', array=photz_ue3),
                        fits.Column(name='Color_Color_Flag', format='D', array=flagz),
                        fits.Column(name='BV_Flag', format='D', array=bv_flag),
                        fits.Column(name='BVi_Flag', format='D', array=bvi_flag),
                        fits.Column(name='BViz_Flag', format='D', array=bviz_flag),
                        fits.Column(name='Stacking', format='D', array=stacking),
                        fits.Column(name='GOODS_x_pixel', format='I', array=misx),
                        fits.Column(name='GOODS_y_pixel', format='I', array=misy),
                        fits.Column(name='GOODS_b_Usable', format='I', array=flag_usable[0]),
                        fits.Column(name='GOODS_v_Usable', format='I', array=flag_usable[2]),
                        fits.Column(name='GOODS_i_Usable', format='I', array=flag_usable[1]),
                        fits.Column(name='GOODS_z_Usable', format='I', array=flag_usable[3]),
                        fits.Column(name='CANDELS_105_Usable', format='I', array=flag_usable[4]),
                        fits.Column(name='CANDELS_125_Usable', format='I', array=flag_usable[5]),
                        fits.Column(name='CANDELS_160_Usable', format='I', array=flag_usable[6]),
                        fits.Column(name='HUGS_Usable', format='I', array=flag_usable[7]),
                        fits.Column(name='GOODS_b_Flag', format='I', array=phot[0][0]),
                        fits.Column(name='GOODS_b_RA', format='D', array=phot[0][1]),
                        fits.Column(name='GOODS_b_Dec', format='D', array=phot[0][2]),
                        fits.Column(name='GOODS_b_MagAuto', format='D', array=phot[0][3]), 
                        fits.Column(name='GOODS_b_MagErrAuto', format='D', array=phot[0][4]),
                        fits.Column(name='GOODS_b_FluxAuto', format='D', array=phot[0][5]),
                        fits.Column(name='GOODS_b_FluxErrAuto', format='D', array=phot[0][6]),
                        fits.Column(name='GOODS_b_JanskyAuto', format='D', array=phot[0][7]),
                        fits.Column(name='GOODS_b_JanskyErrAuto', format='D', array=phot[0][8]),
                        fits.Column(name='GOODS_b_MagAper', format='D', array=phot[0][9]), 
                        fits.Column(name='GOODS_b_MagErrAper', format='D', array=phot[0][10]),
                        fits.Column(name='GOODS_b_FluxAper', format='D', array=phot[0][11]),
                        fits.Column(name='GOODS_b_FluxErrAper', format='D', array=phot[0][12]),
                        fits.Column(name='GOODS_b_JanskyAper', format='D', array=phot[0][13]),
                        fits.Column(name='GOODS_b_JanskyErrAper', format='D', array=phot[0][14]),
                        fits.Column(name='GOODS_v_Flag', format='I', array=phot[1][0]),
                        fits.Column(name='GOODS_v_RA', format='D', array=phot[1][1]),
                        fits.Column(name='GOODS_v_Dec', format='D', array=phot[1][2]),
                        fits.Column(name='GOODS_v_MagAuto', format='D', array=phot[1][3]),
                        fits.Column(name='GOODS_v_MagErrAuto', format='D', array=phot[1][4]),
                        fits.Column(name='GOODS_v_FluxAuto', format='D', array=phot[1][5]),
                        fits.Column(name='GOODS_v_FluxErrAuto', format='D', array=phot[1][6]),
                        fits.Column(name='GOODS_v_JanskyAuto', format='D', array=phot[1][7]),
                        fits.Column(name='GOODS_v_JanskyErrAuto', format='D', array=phot[1][8]),
                        fits.Column(name='GOODS_v_MagAper', format='D', array=phot[1][9]), 
                        fits.Column(name='GOODS_v_MagErrAper', format='D', array=phot[1][10]),
                        fits.Column(name='GOODS_v_FluxAper', format='D', array=phot[1][11]),
                        fits.Column(name='GOODS_v_FluxErrAper', format='D', array=phot[1][12]),
                        fits.Column(name='GOODS_v_JanskyAper', format='D', array=phot[1][13]),
                        fits.Column(name='GOODS_v_JanskyErrAper', format='D', array=phot[1][14]),
                        fits.Column(name='GOODS_i_Flag', format='I', array=phot[2][0]),
                        fits.Column(name='GOODS_i_RA', format='D', array=phot[2][1]),
                        fits.Column(name='GOODS_i_Dec', format='D', array=phot[2][2]),
                        fits.Column(name='GOODS_i_MagAuto', format='D', array=phot[2][3]),
                        fits.Column(name='GOODS_i_MagErrAuto', format='D', array=phot[2][4]),
                        fits.Column(name='GOODS_i_FluxAuto', format='D', array=phot[2][5]),
                        fits.Column(name='GOODS_i_FluxErrAuto', format='D', array=phot[2][6]),
                        fits.Column(name='GOODS_i_JanskyAuto', format='D', array=phot[2][7]),
                        fits.Column(name='GOODS_i_JanskyErrAuto', format='D', array=phot[2][8]),
                        fits.Column(name='GOODS_i_MagAper', format='D', array=phot[2][9]), 
                        fits.Column(name='GOODS_i_MagErrAper', format='D', array=phot[2][10]),
                        fits.Column(name='GOODS_i_FluxAper', format='D', array=phot[2][11]),
                        fits.Column(name='GOODS_i_FluxErrAper', format='D', array=phot[2][12]),
                        fits.Column(name='GOODS_i_JanskyAper', format='D', array=phot[2][13]),
                        fits.Column(name='GOODS_i_JanskyErrAper', format='D', array=phot[2][14]),
                        fits.Column(name='GOODS_z_Flag', format='I', array=phot[3][0]),
                        fits.Column(name='GOODS_z_RA', format='D', array=phot[3][1]),
                        fits.Column(name='GOODS_z_Dec', format='D', array=phot[3][2]),
                        fits.Column(name='GOODS_z_MagAuto', format='D', array=phot[3][3]),
                        fits.Column(name='GOODS_z_MagErrAuto', format='D', array=phot[3][4]),
                        fits.Column(name='GOODS_z_FluxAuto', format='D', array=phot[3][5]),
                        fits.Column(name='GOODS_z_FluxErrAuto', format='D', array=phot[3][6]),
                        fits.Column(name='GOODS_z_JanskyAuto', format='D', array=phot[3][7]),
                        fits.Column(name='GOODS_z_JanskyErrAuto', format='D', array=phot[3][8]),
                        fits.Column(name='GOODS_z_MagAper', format='D', array=phot[3][9]), 
                        fits.Column(name='GOODS_z_MagErrAper', format='D', array=phot[3][10]),
                        fits.Column(name='GOODS_z_FluxAper', format='D', array=phot[3][11]),
                        fits.Column(name='GOODS_z_FluxErrAper', format='D', array=phot[3][12]),
                        fits.Column(name='GOODS_z_JanskyAper', format='D', array=phot[3][13]),
                        fits.Column(name='GOODS_z_JanskyErrAper', format='D', array=phot[3][14]),
                        fits.Column(name='CANDELS_105_Flag', format='I', array=phot[4][0]),
                        fits.Column(name='CANDELS_105_RA', format='D', array=phot[4][1]),
                        fits.Column(name='CANDELS_105_Dec', format='D', array=phot[4][2]),
                        fits.Column(name='CANDELS_105_MagAuto', format='D', array=phot[4][3]),
                        fits.Column(name='CANDELS_105_MagErrAuto', format='D', array=phot[4][4]),
                        fits.Column(name='CANDELS_105_FluxAuto', format='D', array=phot[4][5]),
                        fits.Column(name='CANDELS_105_FluxErrAuto', format='D', array=phot[4][6]),
                        fits.Column(name='CANDELS_105_JanskyAuto', format='D', array=phot[4][7]),
                        fits.Column(name='CANDELS_105_JanskyErrAuto', format='D', array=phot[4][8]),
                        fits.Column(name='CANDELS_105_MagAper', format='D', array=phot[4][9]), 
                        fits.Column(name='CANDELS_105_MagErrAper', format='D', array=phot[4][10]),
                        fits.Column(name='CANDELS_105_FluxAper', format='D', array=phot[4][11]),
                        fits.Column(name='CANDELS_105_FluxErrAper', format='D', array=phot[4][12]),
                        fits.Column(name='CANDELS_105_JanskyAper', format='D', array=phot[4][13]),
                        fits.Column(name='CANDELS_105_JanskyErrAper', format='D', array=phot[4][14]),
                        fits.Column(name='CANDELS_125_Flag', format='I', array=phot[5][0]),
                        fits.Column(name='CANDELS_125_RA', format='D', array=phot[5][1]),
                        fits.Column(name='CANDELS_125_Dec', format='D', array=phot[5][2]),
                        fits.Column(name='CANDELS_125_MagAuto', format='D', array=phot[5][3]),
                        fits.Column(name='CANDELS_125_MagErrAuto', format='D', array=phot[5][4]),
                        fits.Column(name='CANDELS_125_FluxAuto', format='D', array=phot[5][5]),
                        fits.Column(name='CANDELS_125_FluxErrAuto', format='D', array=phot[5][6]),
                        fits.Column(name='CANDELS_125_JanskyAuto', format='D', array=phot[5][7]),
                        fits.Column(name='CANDELS_125_JanskyErrAuto', format='D', array=phot[5][8]),
                        fits.Column(name='CANDELS_125_MagAper', format='D', array=phot[5][9]), 
                        fits.Column(name='CANDELS_125_MagErrAper', format='D', array=phot[5][10]),
                        fits.Column(name='CANDELS_125_FluxAper', format='D', array=phot[5][11]),
                        fits.Column(name='CANDELS_125_FluxErrAper', format='D', array=phot[5][12]),
                        fits.Column(name='CANDELS_125_JanskyAper', format='D', array=phot[5][13]),
                        fits.Column(name='CANDELS_125_JanskyErrAper', format='D', array=phot[5][14]),
                        fits.Column(name='CANDELS_160_Flag', format='I', array=phot[6][0]),
                        fits.Column(name='CANDELS_160_RA', format='D', array=phot[6][1]),
                        fits.Column(name='CANDELS_160_Dec', format='D', array=phot[6][2]),
                        fits.Column(name='CANDELS_160_MagAuto', format='D', array=phot[6][3]),
                        fits.Column(name='CANDELS_160_MagErrAuto', format='D', array=phot[6][4]),
                        fits.Column(name='CANDELS_160_FluxAuto', format='D', array=phot[6][5]),
                        fits.Column(name='CANDELS_160_FluxErrAuto', format='D', array=phot[6][6]),
                        fits.Column(name='CANDELS_160_JanskyAuto', format='D', array=phot[6][7]),
                        fits.Column(name='CANDELS_160_JanskyErrAuto', format='D', array=phot[6][8]),
                        fits.Column(name='CANDELS_160_MagAper', format='D', array=phot[6][9]), 
                        fits.Column(name='CANDELS_160_MagErrAper', format='D', array=phot[6][10]),
                        fits.Column(name='CANDELS_160_FluxAper', format='D', array=phot[6][11]),
                        fits.Column(name='CANDELS_160_FluxErrAper', format='D', array=phot[6][12]),
                        fits.Column(name='CANDELS_160_JanskyAper', format='D', array=phot[6][13]),
                        fits.Column(name='CANDELS_160_JanskyErrAper', format='D', array=phot[6][14]),
                        fits.Column(name='HUGS_Flag', format='I', array=phot[7][0]),
                        fits.Column(name='HUGS_RA', format='D', array=phot[7][1]),
                        fits.Column(name='HUGS_Dec', format='D', array=phot[7][2]),
                        fits.Column(name='HUGS_MagAuto', format='D', array=phot[7][3]),
                        fits.Column(name='HUGS_MagErrAuto', format='D', array=phot[7][4]),
                        fits.Column(name='HUGS_FluxAuto', format='D', array=phot[7][5]),
                        fits.Column(name='HUGS_FluxErrAuto', format='D', array=phot[7][6]),
                        fits.Column(name='HUGS_JanskyAuto', format='D', array=phot[7][7]),
                        fits.Column(name='HUGS_JanskyErrAuto', format='D', array=phot[7][8]),
                        fits.Column(name='HUGS_MagAper', format='D', array=phot[7][9]), 
                        fits.Column(name='HUGS_MagErrAper', format='D', array=phot[7][10]),
                        fits.Column(name='HUGS_FluxAper', format='D', array=phot[7][11]),
                        fits.Column(name='HUGS_FluxErrAper', format='D', array=phot[7][12]),
                        fits.Column(name='HUGS_JanskyAper', format='D', array=phot[7][13]),
                        fits.Column(name='HUGS_JanskyErrAper', format='D', array=phot[7][14]),
                        fits.Column(name='Spitzer_ch1_Flag', format='I', array=phot[8][0]),
                        fits.Column(name='Spitzer_ch1_RA', format='D', array=phot[8][1]),
                        fits.Column(name='Spitzer_ch1_Dec', format='D', array=phot[8][2]),
                        fits.Column(name='Spitzer_ch1_MagAuto', format='D', array=phot[8][3]),
                        fits.Column(name='Spitzer_ch1_MagErrAuto', format='D', array=phot[8][4]),
                        fits.Column(name='Spitzer_ch1_FluxAuto', format='D', array=phot[8][5]),
                        fits.Column(name='Spitzer_ch1_FluxErrAuto', format='D', array=phot[8][6]),
                        fits.Column(name='Spitzer_ch1_JanskyAuto', format='D', array=phot[8][7]),
                        fits.Column(name='Spitzer_ch1_JanskyErrAuto', format='D', array=phot[8][8]),
                        fits.Column(name='Spitzer_ch1_MagAper', format='D', array=phot[8][9]), 
                        fits.Column(name='Spitzer_ch1_MagErrAper', format='D', array=phot[8][10]),
                        fits.Column(name='Spitzer_ch1_FluxAper', format='D', array=phot[8][11]),
                        fits.Column(name='Spitzer_ch1_FluxErrAper', format='D', array=phot[8][12]),
                        fits.Column(name='Spitzer_ch1_JanskyAper', format='D', array=phot[8][13]),
                        fits.Column(name='Spitzer_ch1_JanskyErrAper', format='D', array=phot[8][14]),
                        fits.Column(name='Spitzer_ch2_Flag', format='I', array=phot[9][0]),
                        fits.Column(name='Spitzer_ch2_RA', format='D', array=phot[9][1]),
                        fits.Column(name='Spitzer_ch2_Dec', format='D', array=phot[9][2]),
                        fits.Column(name='Spitzer_ch2_MagAuto', format='D', array=phot[9][3]),
                        fits.Column(name='Spitzer_ch2_MagErrAuto', format='D', array=phot[9][4]),
                        fits.Column(name='Spitzer_ch2_FluxAuto', format='D', array=phot[9][5]),
                        fits.Column(name='Spitzer_ch2_FluxErrAuto', format='D', array=phot[9][6]),
                        fits.Column(name='Spitzer_ch2_JanskyAuto', format='D', array=phot[9][7]),
                        fits.Column(name='Spitzer_ch2_JanskyErrAuto', format='D', array=phot[9][8]),
                        fits.Column(name='Spitzer_ch2_MagAper', format='D', array=phot[9][9]), 
                        fits.Column(name='Spitzer_ch2_MagErrAper', format='D', array=phot[9][10]),
                        fits.Column(name='Spitzer_ch2_FluxAper', format='D', array=phot[9][11]),
                        fits.Column(name='Spitzer_ch2_FluxErrAper', format='D', array=phot[9][12]),
                        fits.Column(name='Spitzer_ch2_JanskyAper', format='D', array=phot[9][13]),
                        fits.Column(name='Spitzer_ch2_JanskyErrAper', format='D', array=phot[9][14]),
                        fits.Column(name='Spitzer_Aper_Flag', format='D', array=spitzer_flag)])
orig_table = big_table
orig_cols = orig_table.columns
hdu = fits.BinTableHDU.from_columns(big_table.columns + new_cols)
os.system('rm -f ../data/catalogues/chandra_7ms/big_table.fits')
hdu.writeto('/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits')